In [1]:
# In this scenario we are going to grab the info from excel files.
'''
We need:
1. Prices in different currencies
2. List with Pusblisher names.
3. Clean Publisher Name
4. Check Suppliers, currencies, publishers, discount, shipping costs
5. Check Suppliers special discounts
6. Check restrictions
7. Get landed Cost
8. List of ISBN to ananlyse.

'''

'\nWe need:\n1. Prices in different currencies\n2. List with Pusblisher names.\n3. Clean Publisher Name\n3. Prices in different currencies\n4. Check Suppliers, currencies, publishers, discount, shipping costs\n5. Check Suppliers special discounts\n6. Check restrictions\n7. Get landed Cost\n8. List of ISBN to ananlyse.\n\n'

In [2]:
# 0. Import Libraries and credentials
import pandas as pd
import pyodbc
import numpy as np
import sqlCredentials as sql

In [3]:
# 0. Get databse connection
# Let's pick all the titles gwe can get from the pricing matrix.

prod_db = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'Server=52.86.56.66;'
    'Database=BookXCenterProduction;'
    'UID='+sql.username+';'
    'PWD='+sql.password+';'
    'Trusted_connection=no;'
)

prouc_db = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'Server=52.86.56.66;'
    'Database=PROCUREMENTDB;'
    'UID='+sql.username+';'
    'PWD='+sql.password+';'
    'Trusted_connection=no;'
)

In [4]:
#1. Prices at different currencies
#this time we are going to use all the titles
list_price_sql = """
SELECT *
FROM Isbn.ListPrice
"""

list_price = pd.read_sql(list_price_sql, prod_db)
list_price.head()

,Id,Isbn,Currency,Price,PriceSource,PriceLastRefreshed,Timestamp
0,1,9780030990014,USD,63.27,NielsenFtp,2020-03-04 14:23:40.0125496,2020-03-26 21:06:05.363333
1,2,9780030997105,USD,27.80,NielsenFtp,2020-03-04 14:23:40.0125496,2020-03-26 21:06:05.363333
2,3,9780041900071,GBP,100.00,GoldenList,2020-03-26 18:00:01.1100000,2020-03-26 21:06:05.363333
3,4,9780043350492,GBP,42.99,GoldenList,2020-03-26 18:00:01.1100000,2020-03-26 21:06:05.363333
4,5,9780044450191,GBP,100.00,GoldenList,2020-03-26 18:00:01.1100000,2020-03-26 21:06:05.363333


In [5]:
list_price.shape

(1747846, 7)

In [6]:
list_price['PriceSource'].unique()

array(['NielsenFtp', 'GoldenList', 'GardnersInventory',
       'BertramsInventory', 'BookDepotInventory', 'TitlePageScraper',
       'BowkerNepModel', 'SAPPurchaseOrder', 'BowkerApiModel',
       'PearsonCanadaScraperModel', 'AbePolandScraper'], dtype=object)

In [7]:
list_price.drop(columns = ['Id', 'PriceSource', 'PriceLastRefreshed', 'Timestamp'], inplace = True)
list_price.head()

,Isbn,Currency,Price
0,9780030990014,USD,63.27
1,9780030997105,USD,27.80
2,9780041900071,GBP,100.00
3,9780043350492,GBP,42.99
4,9780044450191,GBP,100.00


In [8]:
bilblo_sql = """
SELECT *
FROM Isbn.Bibliography
"""
biblio = pd.read_sql(bilblo_sql, prod_db)
biblio.head()

,Isbn,Title,TitleSource,TitleLastRefreshed,Publisher,PublisherSource,PublisherLastRefreshed,Author,AuthorSource,AuthorLastRefreshed,PublicationDate,PublicationDateSource,PublicationDateLastRefreshed,Copyright,CopyrightSource,CopyrightLastRefreshed,Binding,BindingSource,BindingLastRefreshed,Timestamp
0,9780000002402,None,None,None,LWW,GoldenList,2020-03-26 18:00:01.1100000,None,None,None,None,None,None,None,None,None,None,None,None,2020-03-26 21:04:29.987
1,9780000002679,None,None,None,LWW,GoldenList,2020-03-26 18:00:01.1100000,None,None,None,None,None,None,None,None,None,None,None,None,2020-03-26 21:04:29.987
2,9780000003102,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2020-03-26 21:04:29.987
3,9780000003201,None,None,None,LWW,GoldenList,2020-03-26 18:00:01.1100000,None,None,None,None,None,None,None,None,None,None,None,None,2020-03-26 21:04:29.987
4,9780000006493,None,None,None,LWW,GoldenList,2020-03-26 18:00:01.1100000,None,None,None,None,None,None,None,None,None,None,None,None,2020-03-26 21:04:29.987


In [9]:
sup_list = pd.merge(list_price, biblio, how='left', on = 'Isbn' )

In [10]:
sup_list.head()

,Isbn,Currency,Price,Title,TitleSource,TitleLastRefreshed,Publisher,PublisherSource,PublisherLastRefreshed,Author,...,PublicationDate,PublicationDateSource,PublicationDateLastRefreshed,Copyright,CopyrightSource,CopyrightLastRefreshed,Binding,BindingSource,BindingLastRefreshed,Timestamp
0,9780030990014,USD,63.27,Comparative International Accounting,NielsenFtp,2020-03-04 14:23:40.0125496,HOLT MCDOUGAL,NielsenFtp,2020-03-04 14:23:40.0125496,David Alexander,...,19971231,NielsenFtp,2020-03-04 14:23:40.0125496,None,None,None,Paperback / softback,NielsenFtp,2020-03-04 14:23:40.0125496,2020-03-26 21:04:29.987
1,9780030997105,USD,27.80,Holt Traditions Warriner's Handbook: Developme...,NielsenFtp,2020-03-04 14:23:40.0125496,HOLT MCDOUGAL,NielsenFtp,2020-03-04 14:23:40.0125496,Holt Rinehart and Winston,...,20080101,NielsenFtp,2020-03-04 14:23:40.0125496,None,None,None,Paperback / softback,NielsenFtp,2020-03-04 14:23:40.0125496,2020-03-26 21:04:29.987
2,9780041900071,GBP,100.00,None,None,None,TF,GoldenList,2020-03-26 18:00:01.1100000,None,...,None,None,None,None,None,None,None,None,None,2020-03-26 21:04:29.987
3,9780043350492,GBP,42.99,None,None,None,TF,GoldenList,2020-03-26 18:00:01.1100000,None,...,None,None,None,None,None,None,None,None,None,2020-03-26 21:04:29.987
4,9780044450191,GBP,100.00,None,None,None,TF,GoldenList,2020-03-26 18:00:01.1100000,None,...,None,None,None,None,None,None,None,None,None,2020-03-26 21:04:29.987


In [11]:
sup_list.columns

Index(['Isbn', 'Currency', 'Price', 'Title', 'TitleSource',
       'TitleLastRefreshed', 'Publisher', 'PublisherSource',
       'PublisherLastRefreshed', 'Author', 'AuthorSource',
       'AuthorLastRefreshed', 'PublicationDate', 'PublicationDateSource',
       'PublicationDateLastRefreshed', 'Copyright', 'CopyrightSource',
       'CopyrightLastRefreshed', 'Binding', 'BindingSource',
       'BindingLastRefreshed', 'Timestamp'],
      dtype='object')

In [12]:
sup_list = sup_list[['Isbn','Currency', 'Price','Title', 'Publisher', 'Author']]

In [13]:
sup_list.head()

,Isbn,Currency,Price,Title,Publisher,Author
0,9780030990014,USD,63.27,Comparative International Accounting,HOLT MCDOUGAL,David Alexander
1,9780030997105,USD,27.80,Holt Traditions Warriner's Handbook: Developme...,HOLT MCDOUGAL,Holt Rinehart and Winston
2,9780041900071,GBP,100.00,None,TF,None
3,9780043350492,GBP,42.99,None,TF,None
4,9780044450191,GBP,100.00,None,TF,None


In [14]:
sup_list['Publisher'].unique()

array(['HOLT MCDOUGAL', 'TF', None, ..., 'Hindustan Book Agency',
       'Mr Vivek Mehra', 'Woodhead Publishing India Pvt Ltd'],
      dtype=object)

In [15]:
sup_list = sup_list.fillna('N/A')
sup_list.isna().sum()

Isbn         0
Currency     0
Price        0
Title        0
Publisher    0
Author       0
dtype: int64

In [16]:
# 3. Clean publisher
dictio_sql = """
SELECT *
FROM dbo.PublisherDictionary
"""

dictio = pd.read_sql(dictio_sql, prouc_db)
dictio.head()

,PublisherLong,PublisherShort
0,"""Abrams, Inc.""",ABRAMS
1,"""Addison-Wesley Longman, Incorporated""",PEARSON
2,"""Addison-Wesley Longman, Limited""",PEARSON
3,"""Allyn & Bacon, Incorporated""",PEARSON
4,"""American Occupational Therapy Association, In...","American Occupational Therapy Association, Inc..."


In [17]:
def clean_up(phrase):
    phrase = phrase.lstrip()
    phrase = phrase.rstrip()
    phrase = phrase.upper()
    return phrase

In [18]:
for col in list(dictio.columns):
    dictio[col] = dictio.apply(lambda x: clean_up(x[col]), axis =1)

In [19]:
dictio.head()

,PublisherLong,PublisherShort
0,"""ABRAMS, INC.""",ABRAMS
1,"""ADDISON-WESLEY LONGMAN, INCORPORATED""",PEARSON
2,"""ADDISON-WESLEY LONGMAN, LIMITED""",PEARSON
3,"""ALLYN & BACON, INCORPORATED""",PEARSON
4,"""AMERICAN OCCUPATIONAL THERAPY ASSOCIATION, IN...","AMERICAN OCCUPATIONAL THERAPY ASSOCIATION, INC..."


In [20]:
for col in list(sup_list.drop(columns = 'Price').columns):
    sup_list[col] = sup_list.apply(lambda x: clean_up(x[col]), axis =1)

In [21]:
sup_list.head()

,Isbn,Currency,Price,Title,Publisher,Author
0,9780030990014,USD,63.27,COMPARATIVE INTERNATIONAL ACCOUNTING,HOLT MCDOUGAL,DAVID ALEXANDER
1,9780030997105,USD,27.80,HOLT TRADITIONS WARRINER'S HANDBOOK: DEVELOPME...,HOLT MCDOUGAL,HOLT RINEHART AND WINSTON
2,9780041900071,GBP,100.00,N/A,TF,N/A
3,9780043350492,GBP,42.99,N/A,TF,N/A
4,9780044450191,GBP,100.00,N/A,TF,N/A


In [22]:
# Merge on Publisher
sup_list2 = pd.merge(sup_list, dictio, how='left', left_on = 'Publisher', right_on = 'PublisherLong' )

In [23]:
sup_list2.head()

,Isbn,Currency,Price,Title,Publisher,Author,PublisherLong,PublisherShort
0,9780030990014,USD,63.27,COMPARATIVE INTERNATIONAL ACCOUNTING,HOLT MCDOUGAL,DAVID ALEXANDER,HOLT MCDOUGAL,HOLT MCDOUGAL
1,9780030997105,USD,27.80,HOLT TRADITIONS WARRINER'S HANDBOOK: DEVELOPME...,HOLT MCDOUGAL,HOLT RINEHART AND WINSTON,HOLT MCDOUGAL,HOLT MCDOUGAL
2,9780041900071,GBP,100.00,N/A,TF,N/A,NaN,NaN
3,9780043350492,GBP,42.99,N/A,TF,N/A,NaN,NaN
4,9780044450191,GBP,100.00,N/A,TF,N/A,NaN,NaN


In [29]:
# 4. get suppliers discount
sup_pub = pd.read_excel('suppliers.xlsx',  sheet_name='pub')
sup_pub.head()

,Supplier,Publisher,Currency,Discount,Shipping Cost,max quantity,Min,Region
0,COINFO,3D TOTAL,AUD,0.22,3.3,10000,NaN,AUSTRALIA
1,COINFO,ABC CLIO,AUD,0.30,3.3,10000,NaN,AUSTRALIA
2,COINFO,AIAA,AUD,0.30,3.3,10000,NaN,AUSTRALIA
3,COINFO,ALIEN & UNWIN,AUD,0.33,3.3,10000,NaN,AUSTRALIA
4,COINFO,Amer Psych Ass.,AUD,0.33,3.3,10000,NaN,AUSTRALIA


In [30]:
for col in ['Supplier', 'Publisher', 'Currency', 'Region']:
    sup_pub[col] = sup_pub.apply(lambda x: clean_up(x[col]), axis =1)

In [31]:
sup_list3 = pd.merge(sup_list2, sup_pub, how='inner', left_on = ['Currency', 'PublisherShort'], right_on = ['Currency', 'Publisher'] )

In [32]:
sup_list3

,Isbn,Currency,Price,Title,Publisher_x,Author,PublisherLong,PublisherShort,Supplier,Publisher_y,Discount,Shipping Cost,max quantity,Min,Region
0,9780049400870,GBP,27.99,N/A,T&F,N/A,T&F,T&F,ABE,T&F,0.35,3.47,10000,NaN,EUROPE
1,9780049421721,GBP,120.00,N/A,T&F,N/A,T&F,T&F,ABE,T&F,0.35,3.47,10000,NaN,EUROPE
2,9780044459514,GBP,32.50,N/A,T&F,N/A,T&F,T&F,ABE,T&F,0.35,3.47,10000,NaN,EUROPE
3,9780049421844,GBP,120.00,N/A,T&F,N/A,T&F,T&F,ABE,T&F,0.35,3.47,10000,NaN,EUROPE
4,9780044452928,GBP,99.99,N/A,T&F,N/A,T&F,T&F,ABE,T&F,0.35,3.47,10000,NaN,EUROPE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354658,9789027213204,AUD,86.95,INTERLANGUAGE: FORTY YEARS LATER,JOHN BENJAMINS PUBLISHING,ZHAOHONG HAN,JOHN BENJAMINS PUBLISHING,JOHN BENJAMINS PUBLISHING,COINFO,JOHN BENJAMINS PUBLISHING,0.30,3.30,10000,NaN,AUSTRALIA
354659,9789027232618,AUD,54.95,BBI COMBINATORY DICTIONARY OF ENGLISH,JOHN BENJAMINS PUBLISHING,MORTON BENSON,JOHN BENJAMINS PUBLISHING,JOHN BENJAMINS PUBLISHING,COINFO,JOHN BENJAMINS PUBLISHING,0.30,3.30,10000,NaN,AUSTRALIA
354660,9789027232656,AUD,71.95,INTRODUCING SOCIOLINGUISTICS: <STRONG>SECOND E...,JOHN BENJAMINS PUBLISHING,RAJEND MESTHRIE,JOHN BENJAMINS PUBLISHING,JOHN BENJAMINS PUBLISHING,COINFO,JOHN BENJAMINS PUBLISHING,0.30,3.30,10000,NaN,AUSTRALIA
354661,9789027241801,AUD,81.95,INCOMPLETE ACQUISITION IN BILINGUALISM: RE-EXA...,JOHN BENJAMINS PUBLISHING,SILVINA MONTRUL,JOHN BENJAMINS PUBLISHING,JOHN BENJAMINS PUBLISHING,COINFO,JOHN BENJAMINS PUBLISHING,0.30,3.30,10000,NaN,AUSTRALIA


In [34]:
sup_list3.drop(columns = ['Publisher_x', 'PublisherLong', 'Publisher_y'], inplace = True)

In [35]:
sup_list3.head()

,Isbn,Currency,Price,Title,Author,PublisherShort,Supplier,Discount,Shipping Cost,max quantity,Min,Region
0,9780049400870,GBP,27.99,N/A,N/A,T&F,ABE,0.35,3.47,10000,NaN,EUROPE
1,9780049421721,GBP,120.00,N/A,N/A,T&F,ABE,0.35,3.47,10000,NaN,EUROPE
2,9780044459514,GBP,32.50,N/A,N/A,T&F,ABE,0.35,3.47,10000,NaN,EUROPE
3,9780049421844,GBP,120.00,N/A,N/A,T&F,ABE,0.35,3.47,10000,NaN,EUROPE
4,9780044452928,GBP,99.99,N/A,N/A,T&F,ABE,0.35,3.47,10000,NaN,EUROPE


In [36]:
# 5. Check Suppliers special discounts
sp_disc = pd.read_excel("\\fsbx\data\Vendors\. Short Discounted\all_short_discount_list.xlsx", )

FileNotFoundError: [Errno 2] No such file or directory: '\\fsbx\\data\\Vendors\\. Short Discounted\x07ll_short_discount_list.xlsx'